In [2]:
import pandas as pd



In [6]:
df = pd.read_csv('sports_articles.csv', encoding = "ISO-8859-1")


In [9]:
df.head()

,Category,Headline,SubHeading,Link,isResult
0,CYKLING,Italiensk tempotog træder fra både Vingegaard ...,"Filippo Ganna var hurtigst på tirsdagens 25,8 ...",https://www.dr.dk/sporten/cykling/italiensk-te...,True
1,BADMINTON,Dansk mester stopper på landsholdet med bekymr...,"Hans-Kristian Vittinghus mener, at Danmark er ...",https://www.dr.dk/sporten/badminton/dansk-mest...,False
2,ATLETIK,Kæresten fik Simon overtalt - nu kan han kalde...,Danske Simon Jørn Hansen triumferede søndag ef...,https://www.dr.dk/sporten/atletik/kaeresten-fi...,True
